## CMBAgent - OSS Testing

*For testing purposes only, to be removed before opening a PR to* `main` .

In [2]:
# Local model testing supported via https://github.com/acceleratescience/mini-vllm (private Accelerate repo)
from pathlib import Path
from cmbagent.workflows import one_shot
import os

BASE_URL = "http://localhost:8010/v1"
# API_KEY = os.getenv("LOCAL_LLM_API_KEY")
API_KEY = "EMPTY"

from dotenv import load_dotenv
dotenv_path = Path("/scratch/scratch-lxu/venv/cmbagent_env/.env")
load_dotenv(dotenv_path)


local_llm_config = {
    "config_list": [
        {
            "model": "gpt-oss-120b", # Must match model id from endpoint
            "api_key": API_KEY,
            "api_type": "openai",
            "base_url": BASE_URL,
            "price": [0.0, 0.0], # Passed explicitly to ignore warning
        },
    ],
    "cache_seed": None
}


### AG2 ConversableAgent() with `openai/gpt-oss-20b`

Simple chat with a tiny model (`openai/gpt-oss-20b`)to verify ConversableAgent() has OSS

In [3]:
from autogen import UserProxyAgent, ConversableAgent

# Creates the agent that uses the vLLM LM
assistant = ConversableAgent(
    "agent",
    llm_config=local_llm_config,
    system_message="Answer briefly.",
)

# Create the agent that represents the user in the conversation
user_proxy = UserProxyAgent(
    "user",
    code_execution_config=False,
    system_message="",
    human_input_mode="NEVER",
)

# User asks a question
assistant.initiate_chat(
    user_proxy,
    message="What is the meaning of Denario?",
    max_turns=2,
)

agent (to user):

What is the meaning of Denario?

--------------------------------------------------------------------------------
agent (to user):

What is the meaning of Denario?

--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------
user (to agent):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
       Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-oss-120b agent 0.00000             86                244           330
agent (to user):

**Denario** (Spanish) – the term originally refers to the **denarius**, a silver coin used in ancient Rome (≈ 3–4 g of silver). In modern usage it can denote:

1. **A historical Roman coin** – the basic monetary unit of the Roman Republic and early Empire.  
2. **A generic name for a small coin or low‑value currency** in various Roman

ChatResult(chat_id=202150327877313140277316320235895746173, chat_history=[{'content': 'What is the meaning of Denario?', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': 'user'}, {'content': '**Denario** (Spanish) –\u202fthe term originally refers to the **denarius**, a silver coin used in ancient Rome (≈\u202f3–4\u202fg of silver). In modern usage it can denote:\n\n1. **A historical Roman coin** – the basic monetary unit of the Roman Republic and early Empire.  \n2. **A generic name for a small coin or low‑value currency** in various Romance‑language contexts (e.g., old Spanish “denario” meaning “penny” or “cent”).  \n3. **A newspaper title** – several publications in the Spanish‑speaking world are called *El Denario* or simply *Denario*.\n\nSo, “denario” essentially means “a small silver coin” or “a low‑denomination currency unit,” stemming from the ancient Roman denarius.', 'role': 'assistant', 'name': 'agent'}, {'content': '', 'role': 'user', 'name': '

### AG2 ConversableAgent() Tool Calling with OSS

In [4]:
def add(a: float, b: float) -> float:
    return a + b

assistant.register_for_llm(name="add", description="Add two numbers")(add)
assistant.register_for_execution(name="add")(add)

result = assistant.initiate_chat(
    recipient=assistant,
    message="Use the add tool to add 2 and 3. Then reply with just the number.",
    max_turns=3,
)

for msg in result.chat_history:
    if msg.get("tool_calls"):
        print(msg["tool_calls"])

agent (to agent):

Use the add tool to add 2 and 3. Then reply with just the number.

--------------------------------------------------------------------------------
agent (to agent):

Use the add tool to add 2 and 3. Then reply with just the number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


       Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-oss-120b agent 0.00000            175                 49           224
agent (to agent):

{
  "a": 2,
  "b": 3
}

--------------------------------------------------------------------------------
agent (to agent):

{
  "a": 2,
  "b": 3
}

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
       Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-oss-120b agent 0.00000            392                 98           490
agent (to agent):

{
  "a": 2,
  "b": 3
}

--------------------------------------------------------------------------------
agent (to agent):

{
  "a": 2,
  "b": 3
}

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
       Model agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-oss-120b agent 0.00000            651                177         

### OSS with Engineer Only

In [5]:
from pathlib import Path
from cmbagent.workflows import one_shot

local_engineer_config = local_llm_config["config_list"][0]

from dotenv import load_dotenv

load_dotenv()

tmp_path = Path.cwd()

task = """
Plot the graph of y = x from x = 0 to 1
"""

results = one_shot(
    task,

    max_rounds=5,
    agent="engineer",
    engineer_model=local_engineer_config,
    work_dir=str(tmp_path / "one_shot"),
)

_User (to main_cmbagent_chat):


Plot the graph of y = x from x = 0 to 1


--------------------------------------------------------------------------------
_User (to main_cmbagent_chat):


Plot the graph of y = x from x = 0 to 1


--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
       Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-oss-120b engineer 0.00000           1946                900          2846
engineer (to main_cmbagent_chat):

**Code Explanation:**  
The script creates a simple line plot for the function y = x over the interval [0, 1]. It ensures the output directory exists, generates a timestamped filename following the required convention, and saves the figure at 300 dpi without displaying it. After saving, it prints a concise message describing the saved file.

**Python Code:**
```python
"""
Module to plot y = x over the interval [0, 1] and save the figure.
